In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
import seaborn as sns
#cols = [1, 2, 4, 5, 6, 7, 9, 11]
#data = pd.read_csv('train.csv', usecols=cols, true_values=['male'], false_values=['female'])
#data = data.join(pd.get_dummies(data['Embarked'], prefix='Embarked'))
#del data['Embarked']
#data = shuffle(data)
#data = data.astype('float')
#data.fillna(0, inplace=True)
#data['Age_2'] = data['Age']**2
#y = data.pop('Survived')
#m = len(y)
#y_train, X_train = y[:int(0.6*m)], data[:int(0.6*m)]
#y_val, X_val = y[int(0.6*m):int(0.8*m)], data[int(0.6*m):int(0.8*m)]
#y_test, X_test = y[int(0.8*m):], data[int(0.8*m):]
#theta = np.random.rand(X_test.shape[1])

In [50]:
train_df = pd.read_csv('train.csv')
train_df.drop(['PassengerId'], axis=1, inplace=True)
test_df = pd.read_csv('test.csv')
combine = [train_df, test_df]
for df in combine:
    df.drop(['Cabin', 'Ticket'], axis=1, inplace=True)
    df['Title'] = df['Name'].str.extract(' ([a-zA-Z]+)\.', expand=False)
    df.drop('Name', axis=1, inplace=True)
    df['Sex'] = df['Sex'].map({'male':1, 'female':0})


In [51]:
age_mean = train_df[['Title', 'Age']].groupby('Title').mean()
for df in combine:
    for t in age_mean.index:
        df.loc[(df.Age.isnull()) & (df.Title == t), 'Age'] = (age_mean.at[t, 'Age'])

In [52]:
#fill train_df's Embarked NaN
train_df.loc[(train_df.Embarked.isnull()), 'Embarked'] = 'S'

In [53]:
#fill test_df's Fare NaN
test_df.loc[test_df.Fare.isnull(), 'Fare'] = train_df[['Pclass', 'Fare']].groupby('Pclass').mean().loc[3, 'Fare']

In [54]:
for df in combine:
    df['Embarked'] = df['Embarked'].map({'S':1, 'C':3, 'Q':2}).astype(int)
    df['Age'] = df['Age'].apply(lambda x:int(x/16)+1)

In [55]:
train_df[['Embarked', 'Survived']].groupby('Embarked').mean().sort_values(by='Survived')

,Survived
Embarked,
1,0.339009
2,0.389610
3,0.553571


In [56]:
train_df.describe()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.647587,2.492705,0.523008,0.381594,32.204208,1.463524
std,0.486592,0.836071,0.477990,0.843150,1.102743,0.806057,49.693429,0.791500
min,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,2.000000,0.000000,2.000000,0.000000,0.000000,7.910400,1.000000
50%,0.000000,3.000000,1.000000,2.000000,0.000000,0.000000,14.454200,1.000000
75%,1.000000,3.000000,1.000000,3.000000,1.000000,0.000000,31.000000,2.000000
max,1.000000,3.000000,1.000000,6.000000,8.000000,6.000000,512.329200,3.000000


In [57]:
test_df.describe()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,0.636364,2.492823,0.447368,0.392344,35.574673,1.598086
std,120.810458,0.841838,0.481622,0.825956,0.896760,0.981429,55.850822,0.854496
min,892.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,996.250000,1.000000,0.000000,2.000000,0.000000,0.000000,7.895800,1.000000
50%,1100.500000,3.000000,1.000000,2.000000,0.000000,0.000000,14.454200,1.000000
75%,1204.750000,3.000000,1.000000,3.000000,1.000000,0.000000,31.471875,2.000000
max,1309.000000,3.000000,1.000000,5.000000,8.000000,9.000000,512.329200,3.000000


In [58]:
X = train_df.iloc[:, 1:8]
y = train_df.iloc[:, 0]

In [83]:
clf = tree.DecisionTreeClassifier(max_depth=7)
clf.fit(X, y)
clf.score(X, y)

0.87878787878787878

In [84]:
test_df['Survived'] = clf.predict(test_df.iloc[:, 1:8])

In [85]:
test_df[['PassengerId', 'Survived']].to_csv('submit.csv', index=False)

In [65]:
with open("iris.dot", 'w') as f:
    f = tree.export_graphviz(clf, out_file=f)

In [66]:
import os
os.unlink('iris.dot')

In [67]:
import pydotplus 
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = pydotplus.graph_from_dot_data(dot_data) 
graph.write_pdf("iris.pdf") 

ImportError: No module named 'pydotplus'